### Cluster generate

In [ ]:
# ==========================================
# Cell 1: 聚类分析 (Clustering)
# ==========================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import linkage, fcluster
from pathlib import Path
from typing import Tuple, List, Dict
import warnings

# 忽略警告
warnings.filterwarnings('ignore')

# 1. 核心聚类算法
def auto_select_k_and_cluster(X_data: np.ndarray, max_k: int = 10) -> np.ndarray:
    """自动选择最佳K值并返回聚类标签 (基于肘部法则-最大降幅)"""
    n_samples = len(X_data)
    if n_samples < 3:
        return np.zeros(n_samples, dtype=int)

    Z = linkage(X_data, method='ward')
    
    wcss_list = []
    valid_ks = list(range(2, min(max_k, n_samples) + 1))
    
    for k in valid_ks:
        labels = fcluster(Z, k, criterion='maxclust')
        wcss = 0
        for i in range(1, k + 1):
            cluster_points = X_data[labels == i]
            if len(cluster_points) > 0:
                center = cluster_points.mean(axis=0)
                wcss += np.sum((cluster_points - center) ** 2)
        wcss_list.append(wcss)
    
    if len(wcss_list) < 2:
        best_k = valid_ks[0]
    else:
        deltas = [wcss_list[i] - wcss_list[i+1] for i in range(len(wcss_list)-1)]
        best_idx = np.argmax(deltas)
        best_k = valid_ks[best_idx + 1]
    
    return fcluster(Z, best_k, criterion='maxclust') - 1

def run_clustering_single_metric(df_source: pd.DataFrame, l2_name: str, need_scale: bool = False) -> pd.Series:
    """处理单条L2政策"""
    # === 修改点1：严格限制年份 2005-2023 ===
    df = df_source[(df_source['TIME_PERIOD'] >= 2005) & (df_source['TIME_PERIOD'] <= 2023)]
    
    X = df.set_index(['REF_AREA', 'TIME_PERIOD'])[l2_name].unstack().fillna(0)
    
    if X.empty:
        return pd.Series(dtype=int)

    if need_scale:
        scaler = MinMaxScaler()
        if X.values.max() == X.values.min():
            X_vals = X.values
        else:
            X_vals = scaler.fit_transform(X)
    else:
        X_vals = X.values
        
    labels = auto_select_k_and_cluster(X_vals)
    return pd.Series(labels, index=X.index, name='ClusterID')

# 2. 数据 IO
def load_data(data_dir: Path):
    df_b = pd.read_parquet(data_dir / "2-1-country_breadth.parquet")
    df_i = pd.read_parquet(data_dir / "2-1-country_intensity.parquet")
    l2_list = [c for c in df_b.columns if c not in {'REF_AREA', 'TIME_PERIOD'}]
    return df_b, df_i, l2_list

def save_clustered_data(cluster_results: Dict[str, pd.Series], df_source: pd.DataFrame, output_path: Path):
    records = []
    # 同样确保源数据也是 2005-2023，以便保存时一致
    df_source_filtered = df_source[(df_source['TIME_PERIOD'] >= 2005) & (df_source['TIME_PERIOD'] <= 2023)]
    
    for l2, clusters in cluster_results.items():
        if clusters.empty: continue
        
        sub = df_source_filtered[['REF_AREA', 'TIME_PERIOD', l2]].copy()
        sub.columns = ['国家', '年份', '占比'] # 统一列名
        sub['聚类ID'] = sub['国家'].map(clusters)
        sub['L2政策'] = l2
        sub['L2政策中文名'] = l2 # 占位，确保列存在
        
        sub = sub.dropna(subset=['聚类ID'])
        sub['聚类ID'] = sub['聚类ID'].astype(int)
        records.append(sub)
        
    if records:
        final_df = pd.concat(records)
        final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"✅ 已保存: {output_path.name}")

# 3. 执行
base_dir = Path.cwd().parent
data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
print(f"数据目录: {data_dir}")

df_b, df_i, l2_list = load_data(data_dir)

print("正在执行 Breadth 聚类 (2005-2023)...")
b_results = {l2: run_clustering_single_metric(df_b, l2, need_scale=False) for l2 in l2_list}
save_clustered_data(b_results, df_b, data_dir / "3-1-L2_Policy_Clustering_Breadth.csv")

print("正在执行 Intensity 聚类 (2005-2023)...")
i_results = {l2: run_clustering_single_metric(df_i, l2, need_scale=True) for l2 in l2_list}
save_clustered_data(i_results, df_i, data_dir / "3-1-L2_Policy_Clustering_Intensity.csv")

### Table generate

In [ ]:
# ==========================================
# Cell 1: 生成全量数据 -> 模拟 Cell 3 排序 -> 锁定顺序
# ==========================================
import pandas as pd
import numpy as np
import json
from pathlib import Path
from typing import Dict, List, Any

# --- 路径配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
OUTPUT_CSV = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
OUTPUT_JSON = DATA_DIR / "fixed_ppt_order.json"

# --- 1. 特征计算 (来自 Cell 12) ---
def calculate_cluster_metrics(df_sub: pd.DataFrame, is_intensity: bool) -> Dict[int, Dict[str, Any]]:
    th_high = 6.0 if is_intensity else 0.6
    th_med = 3.0 if is_intensity else 0.3
    th_slope = 1.5 if is_intensity else 0.15

    df_calc = df_sub[(df_sub['年份'] >= 2005) & (df_sub['年份'] <= 2023)]
    # 注意：groupby 默认会按 key 排序，这里保证了 trends 的 index 是有序的
    trends = df_calc.groupby(['聚类ID', '年份'])['占比'].mean().unstack()
    
    features = {}
    for cid, row in trends.iterrows():
        ts = row.dropna()
        if len(ts) < 2:
            features[cid] = {'Starting': 'Low', 'Trend': 'Stable', 'Ending': 'Low', 'MeanStart': 0}
            continue
            
        start_val = ts.iloc[:3].mean()
        end_val = ts.iloc[-3].mean()
        slope = end_val - start_val
        std = ts.std()
        
        if start_val > th_high: s_lbl = 'High'
        elif start_val > th_med: s_lbl = 'Medium'
        else: s_lbl = 'Low'
        
        if end_val > th_high: e_lbl = 'High'
        elif end_val > th_med: e_lbl = 'Medium'
        else: e_lbl = 'Low'
        
        if slope > th_slope: t_lbl = 'Rise'
        elif slope < -0.05: t_lbl = 'Decline'
        elif std > (1.0 if is_intensity else 0.1): t_lbl = 'Fluctuate'
        else: t_lbl = 'Stable'
        
        features[cid] = {
            'Starting': s_lbl, 'Trend': t_lbl, 'Ending': e_lbl, 'MeanStart': start_val
        }
    return features

# --- 2. 排序逻辑 (100% 复刻 Cell 3) ---
def sort_clusters_original_logic(sort_df: pd.DataFrame) -> pd.DataFrame:
    """完全使用 Cell 3 的排序 Key 逻辑"""
    orders = {
        'Starting': {'Low': 0, 'Medium': 1, 'High': 2},
        'Ending': {'Low': 0, 'Medium': 1, 'High': 2},
        'Trend': {'Rise': 0, 'Stable': 1, 'Fluctuate': 2, 'Decline': 3}
    }
    
    df = sort_df.copy()
    df['sort_key'] = df.apply(lambda r: (
        orders['Starting'].get(r['Starting'], 99),
        orders['Ending'].get(r['Ending'], 99),
        orders['Trend'].get(r['Trend'], 99),
        r.get('MeanStart', 0)
    ), axis=1)
    
    return df.sort_values('sort_key')

# --- 3. 主执行流程 ---
def process_and_lock_strict():
    print("🚀 开始执行：生成数据 -> 全量排序 -> 锁定 ID")
    
    all_records = []
    
    # === 步骤 A: 生成所有数据 (模拟 Cell 12 的写入过程) ===
    tasks = [
        ("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth", False),
        ("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity", True)
    ]

    for fname, m_type, is_intensity in tasks:
        fpath = DATA_DIR / fname
        if not fpath.exists():
            print(f"⚠️ 跳过文件: {fname}")
            continue

        df_raw = pd.read_csv(fpath, encoding='utf-8-sig')
        
        # 严格按照 Cell 12 的循环顺序生成记录
        for l2 in df_raw['L2政策中文名'].dropna().unique():
            l2_data = df_raw[df_raw['L2政策中文名'] == l2]
            feats_map = calculate_cluster_metrics(l2_data, is_intensity)
            
            # 模拟 Cell 12 的 iterrows 展开
            for _, row in l2_data.iterrows():
                cid = row['聚类ID']
                if cid in feats_map:
                    f = feats_map[cid]
                    all_records.append({
                        'L2政策中文名': l2,
                        '国家': row['国家'],
                        '聚类ID': cid,
                        'Starting': f['Starting'],
                        'Trend': f['Trend'],
                        'Ending': f['Ending'],
                        'MeanStart': f['MeanStart'],
                        'Type': m_type
                    })

    if not all_records:
        print("❌ 没有生成任何数据")
        return

    # === 步骤 B: 构建 DataFrame 并去重 (模拟 Cell 12 的输出) ===
    df_full = pd.DataFrame(all_records).drop_duplicates(subset=['L2政策中文名', '国家', 'Type'])
    
    # 保存 CSV (这是 Cell 3 读取的对象)
    df_full.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"✅ CSV 已保存: {OUTPUT_CSV.name}")

    # === 步骤 C: 模拟 Cell 3 的读取和排序过程 ===
    # 为了绝对严谨，我们直接对 df_full 进行操作，就像 Cell 3 读进来一样
    order_map = {}
    
    # 1. 筛选 metric (Cell 3 是分开跑的，这里我们循环跑)
    for m_type in ['Breadth', 'Intensity']:
        df_metric = df_full[df_full['Type'] == m_type].copy()
        if df_metric.empty: continue
        
        # 2. 全量排序 (Cell 3 的关键步骤: sort_clusters(df_meta))
        # 这会根据 Starting/Ending 等对整个表排序
        df_metric_sorted = sort_clusters_original_logic(df_metric)
        
        # 3. 循环 L2 提取 ID (Cell 3 的绘图循环)
        for l2 in df_metric_sorted['L2政策中文名'].unique():
            # 提取该 L2 下的所有行
            l2_sort_info = df_metric_sorted[df_metric_sorted['L2政策中文名'] == l2]
            
            # Cell 3 里的逻辑: l2_sort_info.sort_values('sort_key')['聚类ID'].unique()
            # 虽然 df_metric_sorted 已经是排好序的，但 Cell 3 里为了保险又 sort 了一次
            # 我们这里也照做，确保行为一致
            final_sorted_df = sort_clusters_original_logic(l2_sort_info)
            sorted_ids = final_sorted_df['聚类ID'].unique().tolist()
            
            order_map[f"{m_type}_{l2}"] = sorted_ids

    # === 步骤 D: 锁定 JSON ===
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(order_map, f, ensure_ascii=False, indent=2)
    print(f"✅ 顺序已严格锁定至 JSON: {OUTPUT_JSON.name}")

# 运行
process_and_lock_strict()

### Wait Human change

### Wait Human change

In [5]:
# ==========================================
# Cell 12: 计算特征并保存文件 (Generate Metadata)
# ==========================================
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Any

# 路径配置
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
OUTPUT_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"

def calculate_cluster_metrics(df_sub: pd.DataFrame, is_intensity: bool) -> Dict[int, Dict[str, Any]]:
    """
    计算指定L2政策下各聚类的形态特征 (High/Low, Rising/Stable等)。
    """
    # 阈值设定
    th_high = 6.0 if is_intensity else 0.6
    th_med = 3.0 if is_intensity else 0.3
    th_slope = 1.5 if is_intensity else 0.15

    # 筛选年份并计算均值曲线
    df_calc = df_sub[(df_sub['年份'] >= 2005) & (df_sub['年份'] <= 2023)]
    trends = df_calc.groupby(['聚类ID', '年份'])['占比'].mean().unstack()
    
    features = {}
    for cid, row in trends.iterrows():
        ts = row.dropna()
        if len(ts) < 2:
            features[cid] = {'Starting': 'Low', 'Trend': 'Stable', 'Ending': 'Low', 'MeanStart': 0}
            continue
            
        start_val = ts.iloc[:3].mean()
        end_val = ts.iloc[-3].mean()
        slope = end_val - start_val
        std = ts.std()
        
        # 判定 Starting
        if start_val > th_high: s_lbl = 'High'
        elif start_val > th_med: s_lbl = 'Medium'
        else: s_lbl = 'Low'
        
        # 判定 Ending
        if end_val > th_high: e_lbl = 'High'
        elif end_val > th_med: e_lbl = 'Medium'
        else: e_lbl = 'Low'
        
        # 判定 Trend
        if slope > th_slope: t_lbl = 'Rise'
        elif slope < -0.05: t_lbl = 'Decline'
        elif std > (1.0 if is_intensity else 0.1): t_lbl = 'Fluctuate'
        else: t_lbl = 'Stable'
        
        features[cid] = {
            'Starting': s_lbl, 'Trend': t_lbl, 'Ending': e_lbl, 'MeanStart': start_val
        }
    return features

def generate_metadata(file_name: str, metric_type: str) -> pd.DataFrame:
    """读取聚类结果，生成包含特征标签的元数据表"""
    file_path = DATA_DIR / file_name
    if not file_path.exists():
        print(f"⚠️ 文件未找到: {file_name}")
        return pd.DataFrame()

    df = pd.read_csv(file_path, encoding='utf-8-sig')
    is_intensity = (metric_type == 'Intensity')
    records = []

    for l2 in df['L2政策中文名'].unique():
        l2_data = df[df['L2政策中文名'] == l2]
        feats_map = calculate_cluster_metrics(l2_data, is_intensity)
        
        # 将特征映射回该聚类的所有国家
        # 为了去重，我们只需保留 (L2, ClusterID) 的唯一组合即可，
        # 但为了后续方便，这里保留国家字段并在返回时去重
        for _, row in l2_data.iterrows():
            cid = row['聚类ID']
            if cid in feats_map:
                f = feats_map[cid]
                records.append({
                    'L2政策中文名': l2,
                    '国家': row['国家'],
                    '聚类ID': cid,
                    'Starting': f['Starting'],
                    'Trend': f['Trend'],
                    'Ending': f['Ending'],
                    'MeanStart': f['MeanStart'],
                    'Type': metric_type
                })
    
    return pd.DataFrame(records).drop_duplicates(subset=['L2政策中文名', '国家', 'Type'])

# --- 主执行流程 ---
print(f"正在处理数据并保存至: {OUTPUT_FILE}")
df_breadth = generate_metadata("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth")
df_intensity = generate_metadata("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity")

if not df_breadth.empty and not df_intensity.empty:
    full_df = pd.concat([df_breadth, df_intensity], ignore_index=True)
    full_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"✅ 成功保存 {len(full_df)} 条特征记录。")
else:
    print("❌ 生成失败，请检查源文件。")

正在处理数据并保存至: f:\Desktop\科研项目\1.负责科研项目\Climate Policy\CAMPF_Supplementary\data\3-2-Automated_Recognition_Mode.csv
✅ 成功保存 1470 条特征记录。


In [3]:
import pandas as pd
from pptx import Presentation
from pathlib import Path
from pptx.util import Inches
from typing import List, Optional
import re

# ==========================================
# 1. 强力清洗函数 (解决横杠不匹配问题)
# ==========================================
def normalize_title_robust(text: str) -> str:
    """
    终极清洗：
    1. 转小写
    2. 把所有类型的横杠 (En-dash, Em-dash) 统一替换为普通减号 '-'
    3. 去除多余空格
    """
    if not isinstance(text, str): return str(text) if text is not None else ""
    
    # 替换各种奇怪的横杠为普通减号
    text = text.replace('\u2013', '-').replace('\u2014', '-').replace('–', '-')
    
    # 替换不换行空格
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    
    # 去除多余空格并转小写
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

def get_slide_title(slide) -> Optional[str]:
    # 优先取标准标题
    if slide.shapes.title and slide.shapes.title.text.strip():
        return slide.shapes.title.text.strip() # 返回原始文本，清洗在后面做
    
    # 回退：取最上面的文本框
    candidates = []
    for shape in slide.shapes:
        if shape.has_text_frame and shape.text_frame.text.strip():
            candidates.append((shape.top, shape.text_frame.text.strip()))
    
    if candidates:
        candidates.sort(key=lambda x: x[0])
        if candidates[0][0] < Inches(2.0):
            return candidates[0][1]
    return None

# ==========================================
# 2. 稳定排序逻辑 (仅基于数值)
# ==========================================
def get_sorted_cluster_ids(df_sub: pd.DataFrame) -> List[int]:
    """
    使用 MeanStart (数值) 作为唯一的排序锚点。
    """
    df_sub = df_sub.copy()
    # 排序键：MeanStart (主), 聚类ID (次)
    df_sub['sort_key'] = df_sub.apply(lambda r: (r.get('MeanStart', 0), r.get('聚类ID', 0)), axis=1)
    return df_sub.sort_values('sort_key')['聚类ID'].unique().tolist()

# ==========================================
# 3. 核心同步逻辑
# ==========================================
def sync_ppt_robust(ppt_path: Path, csv_path: Path, target_type: str) -> None:
    print(f"\n{'='*50}")
    print(f"📂 处理文件: {ppt_path.name}")
    print(f"🎯 目标类型: {target_type}")
    print(f"{'='*50}")
    
    if not ppt_path.exists():
        print("❌ 文件不存在")
        return

    prs = Presentation(ppt_path)
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    
    # === 关键：创建一列标准化的标题用于匹配 ===
    # 这样 CSV 里的 "Buildings – market" 和 PPT 里的 "Buildings - market" 就能匹配上了
    df['Match_Title'] = df['L2政策中文名'].apply(normalize_title_robust)
    
    update_count = 0
    
    for slide_idx, slide in enumerate(prs.slides):
        raw_ppt_title = get_slide_title(slide)
        if not raw_ppt_title: continue
        
        # 标准化 PPT 标题
        clean_ppt_title = normalize_title_robust(raw_ppt_title)
        
        # 1. 尝试匹配
        mask = (df['Match_Title'] == clean_ppt_title) & (df['Type'] == target_type)
        target_rows = df[mask]
        
        if target_rows.empty:
            # 调试：如果是那个出问题的政策，打印一下看为什么没匹配上
            if "market" in clean_ppt_title and "instrument" in clean_ppt_title:
                print(f"⚠️ [未匹配] PPT标题: '{raw_ppt_title}' (清洗后: '{clean_ppt_title}')")
            continue
            
        # 2. 获取稳定的 ID 顺序
        sorted_cids = get_sorted_cluster_ids(target_rows)
        
        # 3. 获取 PPT 标签框 (Left -> Right)
        label_shapes = [s for s in slide.shapes if s.has_text_frame and "+" in s.text_frame.text]
        label_shapes.sort(key=lambda s: (int(s.top / Inches(1)), s.left))
        
        if len(label_shapes) != len(sorted_cids): 
            # 框数量对不上
            if "market" in clean_ppt_title:
                 print(f"⚠️ [数量不符] {raw_ppt_title}: PPT有{len(label_shapes)}个框, CSV有{len(sorted_cids)}个聚类")
            continue
            
        # 4. 逐一更新
        for shape, cid in zip(label_shapes, sorted_cids):
            raw_text = normalize_spaces(shape.text_frame.text)
            parts = raw_text.split('+')
            
            if len(parts) >= 3:
                p1 = parts[0].strip(); p2 = parts[1].strip(); p3_full = parts[2].strip()
                
                final_e = p3_full
                last_paren = p3_full.rfind('(')
                if last_paren != -1:
                    if "countr" in p3_full[last_paren:].lower():
                        final_e = p3_full[:last_paren].strip()
                
                new_s = p1.replace(' Share', '')
                new_t = p2
                new_e = final_e.replace(' Share', '')
                
                # 5. 批量锁定并更新
                # 注意：这里我们用标准化后的 Match_Title 来定位行
                target_idx = df[(df['Match_Title'] == clean_ppt_title) & 
                                (df['Type'] == target_type) & 
                                (df['聚类ID'] == cid)].index
                
                if not target_idx.empty:
                    # 检查并更新
                    old_vals = df.loc[target_idx[0], ['Starting', 'Trend', 'Ending']].values
                    new_vals = [new_s, new_t, new_e]
                    
                    # 只要任何一个字段变了，就更新
                    if list(old_vals) != new_vals:
                        print(f"  🔄 [更新] {raw_ppt_title[:20]}... (ID:{cid})")
                        print(f"     旧: {old_vals}")
                        print(f"     新: {new_vals}")
                        
                        df.loc[target_idx, 'Starting'] = new_s
                        df.loc[target_idx, 'Trend'] = new_t
                        df.loc[target_idx, 'Ending'] = new_e
                        update_count += len(target_idx)

    # 清理辅助列
    if 'Match_Title' in df.columns: del df['Match_Title']
    
    if update_count > 0:
        try:
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f"✅ 保存成功！共更新了 {update_count} 行数据。")
        except PermissionError:
            print("❌ 保存失败！请关闭 Excel 文件！")
    else:
        print("✨ 没有检测到需要更新的内容。")

# ==========================================
# 4. 执行
# ==========================================
def main():
    base_dir = Path.cwd().parent
    data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
    
    ppt_dir = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "Human_catorgy"
    csv_file = data_dir / "3-2-Automated_Recognition_Mode.csv"
    
    # 1. Breadth
    sync_ppt_robust(ppt_dir / "Analysis_Report_Breadth_Editable.pptx", csv_file, "Breadth")
    
    # 2. Intensity
    sync_ppt_robust(ppt_dir / "Analysis_Report_Intensity_Editable.pptx", csv_file, "Intensity")

if __name__ == "__main__":
    main()


📂 处理文件: Analysis_Report_Breadth_Editable.pptx
🎯 目标类型: Breadth


NameError: name 'normalize_spaces' is not defined

In [2]:
import pandas as pd
from pptx import Presentation
from pathlib import Path
from pptx.util import Inches
from typing import List, Optional
import re

# ==========================================
# 1. 强力清洗工具
# ==========================================
def deep_clean(text):
    """
    核弹级清洗：去除所有看不见的字符，统一转成标准格式。
    """
    if pd.isna(text) or text is None:
        return ""
    text = str(text)
    # 替换不换行空格(\xa0), 制表符, 换行符
    text = text.replace('\xa0', ' ').replace('\t', ' ').replace('\n', ' ')
    # 替换各种奇怪的横杠
    text = text.replace('\u2013', '-').replace('\u2014', '-').replace('–', '-')
    # 去除 ' Share' 后缀 (不分大小写)
    text = re.sub(r'(?i)\s*share$', '', text)
    # 压缩多余空格并去除首尾空格
    return re.sub(r'\s+', ' ', text).strip()

def normalize_title_robust(text: str) -> str:
    return deep_clean(text).lower()

def get_slide_title(slide) -> Optional[str]:
    if slide.shapes.title and slide.shapes.title.text.strip():
        return slide.shapes.title.text.strip()
    candidates = []
    for shape in slide.shapes:
        if shape.has_text_frame and shape.text_frame.text.strip():
            candidates.append((shape.top, shape.text_frame.text.strip()))
    if candidates:
        candidates.sort(key=lambda x: x[0])
        if candidates[0][0] < Inches(2.0):
            return candidates[0][1]
    return None

# ==========================================
# 2. 绝对稳定的排序逻辑 (核心修改)
# ==========================================
def get_strictly_stable_ids(df_sub: pd.DataFrame, metric_type: str) -> List[int]:
    """
    【绝对稳定排序】
    不再读取 'Ending' 或 'Trend' 文本进行排序，防止修改文本导致顺序跳变的死循环。
    
    策略：
    1. 先按 Calculated Starting (由 MeanStart 算出来的类别) 分大组。
    2. 组内只按 MeanStart 数值排序。
    这样无论你怎么改文字，排序顺序永远固定。
    """
    df = df_sub.copy()
    
    # 阈值 (参考自绘图代码)
    if metric_type == 'Intensity':
        th_high, th_med = 6.0, 3.0
    else: # Breadth
        th_high, th_med = 0.6, 0.3

    # 动态计算起始等级 (0=Low, 1=Med, 2=High)
    def get_start_rank(val):
        if val > th_high: return 2
        if val > th_med: return 1
        return 0

    df['calc_start_rank'] = df['MeanStart'].apply(get_start_rank)
    
    # 排序键：(起始等级, 起始数值, ID)
    # 注意：这里完全抛弃了 Ending 和 Trend，确保“锚点”不动
    df['sort_key'] = df.apply(lambda r: (
        r['calc_start_rank'], 
        r.get('MeanStart', 0),
        r.get('聚类ID', 0)
    ), axis=1)
    
    return df.sort_values('sort_key')['聚类ID'].unique().tolist()

# ==========================================
# 3. 同步主程序 (带详细调试)
# ==========================================
def sync_ppt_final(ppt_path: Path, csv_path: Path, target_type: str) -> None:
    print(f"\n{'='*50}")
    print(f"🚀 开始处理: {ppt_path.name}")
    print(f"🔒 模式: 绝对稳定排序 (防止死循环)")
    print(f"{'='*50}")
    
    if not ppt_path.exists():
        print("❌ PPT 文件不存在")
        return

    prs = Presentation(ppt_path)
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    
    # 预处理 CSV 标题列
    df['Match_Title'] = df['L2政策中文名'].apply(normalize_title_robust)
    
    update_count = 0
    total_slides_checked = 0
    
    for slide_idx, slide in enumerate(prs.slides):
        raw_ppt_title = get_slide_title(slide)
        if not raw_ppt_title: continue
        
        clean_ppt_title = normalize_title_robust(raw_ppt_title)
        
        # 1. 匹配数据
        mask = (df['Match_Title'] == clean_ppt_title) & (df['Type'] == target_type)
        target_rows = df[mask]
        
        if target_rows.empty: continue
        total_slides_checked += 1
            
        # 2. 获取稳定 ID 列表
        sorted_cids = get_strictly_stable_ids(target_rows, target_type)
        
        # 3. 获取 PPT 文本框 (Top -> Left)
        label_shapes = [s for s in slide.shapes if s.has_text_frame and "+" in s.text_frame.text]
        label_shapes.sort(key=lambda s: (int(s.top / Inches(1)), s.left))
        
        # 4. 数量检查
        if len(label_shapes) != len(sorted_cids):
            if "market" in clean_ppt_title:
                print(f"⚠️ [跳过] {raw_ppt_title}: PPT框数量({len(label_shapes)}) != CSV行数({len(sorted_cids)})")
            continue
            
        # 5. 逐一比对与更新
        for shape, cid in zip(label_shapes, sorted_cids):
            # --- 解析 PPT ---
            raw_text = shape.text_frame.text
            clean_text = deep_clean(raw_text) # "High+Rise+Low (5 countries)"
            
            parts = clean_text.split('+')
            if len(parts) < 3: continue
            
            p1 = parts[0].strip() # Starting
            p2 = parts[1].strip() # Trend
            p3_full = parts[2].strip() # Ending + (Count)
            
            # 提取 Ending (去掉括号部分)
            final_e = p3_full
            if '(' in p3_full:
                 final_e = p3_full.split('(')[0].strip()
            
            # --- 解析 CSV ---
            target_idx = df[(df['Match_Title'] == clean_ppt_title) & 
                            (df['Type'] == target_type) & 
                            (df['聚类ID'] == cid)].index
            
            if target_idx.empty: continue
            
            # 获取当前 CSV 里的值 (并进行同样的深层清洗)
            curr_s = deep_clean(df.loc[target_idx[0], 'Starting'])
            curr_t = deep_clean(df.loc[target_idx[0], 'Trend'])
            curr_e = deep_clean(df.loc[target_idx[0], 'Ending'])
            
            # --- 核心对比 ---
            # 只有当清洗后的内容真的不一样时，才触发更新
            has_change = False
            if curr_s != p1: has_change = True
            if curr_t != p2: has_change = True
            if curr_e != final_e: has_change = True
            
            if has_change:
                print(f" 🔄 [发现差异] {raw_ppt_title[:15]}... (ID:{cid})")
                print(f"    PPT: {p1} + {p2} + {final_e}")
                print(f"    CSV: {curr_s} + {curr_t} + {curr_e}")
                
                df.loc[target_idx, 'Starting'] = p1
                df.loc[target_idx, 'Trend'] = p2
                df.loc[target_idx, 'Ending'] = final_e
                update_count += 1

    # 清理临时列
    if 'Match_Title' in df.columns: del df['Match_Title']
    if 'calc_start_rank' in df.columns: del df['calc_start_rank']
    if 'sort_key' in df.columns: del df['sort_key']
    
    # 保存结果
    if update_count > 0:
        try:
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f"✅ 保存成功！更新了 {update_count} 处。")
        except PermissionError:
            print("❌ 保存失败！请关闭 Excel 文件！")
    else:
        print(f"✨ 检查了 {total_slides_checked} 页，数据完全一致，无需更新！")

# ==========================================
# 4. 执行
# ==========================================
def main():
    base_dir = Path.cwd().parent
    data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
    
    ppt_dir = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "Human_catorgy"
    csv_file = data_dir / "3-2-Automated_Recognition_Mode.csv"
    
    sync_ppt_final(ppt_dir / "Analysis_Report_Breadth_Editable.pptx", csv_file, "Breadth")
    sync_ppt_final(ppt_dir / "Analysis_Report_Intensity_Editable.pptx", csv_file, "Intensity")

if __name__ == "__main__":
    main()


🚀 开始处理: Analysis_Report_Breadth_Editable.pptx
🔒 模式: 绝对稳定排序 (防止死循环)
✨ 检查了 15 页，数据完全一致，无需更新！

🚀 开始处理: Analysis_Report_Intensity_Editable.pptx
🔒 模式: 绝对稳定排序 (防止死循环)
✨ 检查了 15 页，数据完全一致，无需更新！
